In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import neattext.functions as nfx

df=pd.read_csv("emotion_dataset_raw.csv")
df['Clean_Text']=df['Text'].apply(nfx.remove_userhandles)
df['Clean_Text'] = df['Clean_Text'].apply(nfx.remove_stopwords)

x = df['Clean_Text']
y = df['Emotion']
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=42)

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
pipe_lr = Pipeline(steps=[('cv',CountVectorizer()),('lr',LogisticRegression())])
pipe_lr.fit(x_train,y_train)
pipe_lr.score(x_test,y_test)
import joblib
pipeline_file = open("text_emotion.pkl","wb")
joblib.dump(pipe_lr,pipeline_file)
pipeline_file.close()

C:\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.6197547422877946

In [5]:
import joblib
pipeline_file = open("text_emotion.pkl","wb")
joblib.dump(pipe_lr,pipeline_file)
pipeline_file.close()


In [2]:

import pandas as pd
import joblib
import re
import string
import neattext.functions as nfx
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Load dataset
df = pd.read_csv("emotion_dataset_raw.csv")

# Text cleaning function
lemmatizer = WordNetLemmatizer()
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = nfx.remove_userhandles(text)
    text = nfx.remove_stopwords(text)
    text = " ".join([lemmatizer.lemmatize(word) for word in text.split()])
    return text

# Apply cleaning
df['Clean_Text'] = df['Text'].apply(clean_text)

# Features and labels
x = df['Clean_Text']
y = df['Emotion']

# Split data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

# Compute class weights properly
unique_classes = np.unique(y_train)
class_weights = compute_class_weight('balanced', classes=unique_classes, y=y_train)
class_weights_dict = {cls: weight for cls, weight in zip(unique_classes, class_weights)}

# Define pipeline
pipe_svm = Pipeline(steps=[
    ('tfidf', TfidfVectorizer(max_features=7000, ngram_range=(1,3))),
    ('svm', SVC(kernel='linear', C=1.0, class_weight=class_weights_dict, probability=True))
])

# Train model
pipe_svm.fit(x_train, y_train)


# Evaluate model
from sklearn.metrics import accuracy_score, classification_report
y_pred = pipe_svm.predict(x_test)
print("🔹 Accuracy:", accuracy_score(y_test, y_pred))
print("🔹 Classification Report:\n", classification_report(y_test, y_pred))

# Save model
joblib.dump(pipe_svm, "text_emotion_svm_improved.pkl")
print("✅ Improved Model Saved Successfully!")

🔹 Accuracy: 0.581720636137191
🔹 Classification Report:
               precision    recall  f1-score   support

       anger       0.52      0.59      0.56      1283
     disgust       0.20      0.36      0.26       292
        fear       0.72      0.63      0.67      1645
         joy       0.71      0.61      0.66      3311
     neutral       0.44      0.75      0.55       675
     sadness       0.60      0.52      0.55      2015
       shame       0.85      0.94      0.89        36
    surprise       0.45      0.50      0.47      1181

    accuracy                           0.58     10438
   macro avg       0.56      0.61      0.58     10438
weighted avg       0.61      0.58      0.59     10438

✅ Improved Model Saved Successfully!


In [6]:
import pandas as pd
import joblib
import re
import string
import neattext.functions as nfx
from textblob import Word
from autocorrect import Speller
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Load dataset
df = pd.read_csv("emotion_dataset_raw.csv")

# Initialize helpers
lemmatizer = WordNetLemmatizer()
spell = Speller(lang='en')

# Text cleaning function
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'https?://\S+|www\.\S+', '', text)  # Remove URLs
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    text = nfx.remove_userhandles(text)  # Remove @mentions
    text = " ".join([spell(Word(word).lemmatize()) for word in text.split()])  # Spelling correction & lemmatization
    return text

# Apply cleaning
df['Clean_Text'] = df['Text'].apply(clean_text)

# Features and labels
x = df['Clean_Text']
y = df['Emotion']

# Split data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

# Define pipeline with optimized TF-IDF and XGBoost
pipe_xgb = Pipeline(steps=[
    ('tfidf', TfidfVectorizer(max_features=12000, ngram_range=(1,2), sublinear_tf=True)),
    ('xgb', XGBClassifier(n_estimators=200, learning_rate=0.1, max_depth=6, eval_metric='mlogloss'))
])

# Train model
pipe_xgb.fit(x_train, y_train)

# Evaluate model
y_pred = pipe_xgb.predict(x_test)
print("🔹 Accuracy:", accuracy_score(y_test, y_pred))
print("🔹 Classification Report:\n", classification_report(y_test, y_pred))

# Hyperparameter tuning for XGBoost
param_grid = {
    'xgb__n_estimators': [100, 200, 300],
    'xgb__max_depth': [4, 6, 8],
    'xgb__learning_rate': [0.05, 0.1, 0.2]
}

grid_search = GridSearchCV(pipe_xgb, param_grid, cv=3, scoring='accuracy', n_jobs=-1)
grid_search.fit(x_train, y_train)

print("🔹 Best Parameters:", grid_search.best_params_)

# Save model
joblib.dump(grid_search.best_estimator_, "text_emotion_xgb_improved.pkl")
print("✅ Improved Model Saved Successfully!")


ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1 2 3 4 5 6 7], got ['anger' 'disgust' 'fear' 'joy' 'neutral' 'sadness' 'shame' 'surprise']

In [ ]:
import pandas as pd
import joblib
import re
import string
import neattext.functions as nfx
from textblob import Word
from autocorrect import Speller
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

# Load dataset
df = pd.read_csv("emotion_dataset_raw.csv")

# Initialize helpers
lemmatizer = WordNetLemmatizer()
spell = Speller(lang='en')

# Text cleaning function
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'https?://\S+|www\.\S+', '', text)  # Remove URLs
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    text = nfx.remove_userhandles(text)  # Remove @mentions
    text = " ".join([spell(Word(word).lemmatize()) for word in text.split()])  # Spelling correction & lemmatization
    return text

# Apply cleaning
df['Clean_Text'] = df['Text'].apply(clean_text)

# Features and labels
x = df['Clean_Text']
y = df['Emotion']

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split data
x_train, x_test, y_train, y_test = train_test_split(x, y_encoded, test_size=0.3, random_state=42)

# Define pipeline with optimized TF-IDF and XGBoost
pipe_xgb = Pipeline(steps=[
    ('tfidf', TfidfVectorizer(max_features=12000, ngram_range=(1,2), sublinear_tf=True)),
    ('xgb', XGBClassifier(n_estimators=200, learning_rate=0.1, max_depth=6, eval_metric='mlogloss'))
])

# Train model
pipe_xgb.fit(x_train, y_train)

# Evaluate model
y_pred_encoded = pipe_xgb.predict(x_test)
y_pred = label_encoder.inverse_transform(y_pred_encoded)  # Decode predictions

print("🔹 Accuracy:", accuracy_score(y_test, y_pred_encoded))
print("🔹 Classification Report:\n", classification_report(y_test, y_pred_encoded))

# Hyperparameter tuning for XGBoost
param_grid = {
    'xgb__n_estimators': [100, 200, 300],
    'xgb__max_depth': [4, 6, 8],
    'xgb__learning_rate': [0.05, 0.1, 0.2]
}

grid_search = GridSearchCV(pipe_xgb, param_grid, cv=3, scoring='accuracy', n_jobs=-1)
grid_search.fit(x_train, y_train)

print("🔹 Best Parameters:", grid_search.best_params_)

# Save model and label encoder
joblib.dump(grid_search.best_estimator_, "text_emotion_xgb_improved.pkl")
joblib.dump(label_encoder, "label_encoder.pkl")
print("✅ Improved Model and Label Encoder Saved Successfully!")

# Function to predict emotion on new text
def predict_emotion(text):
    text_cleaned = clean_text(text)
    model = joblib.load("text_emotion_xgb_improved.pkl")
    encoder = joblib.load("label_encoder.pkl")
    pred_encoded = model.predict([text_cleaned])
    return encoder.inverse_transform(pred_encoded)[0]

# Example usage
new_text = "I am very happy today!"
pred_emotion = predict_emotion(new_text)
print("Predicted Emotion:", pred_emotion)


🔹 Accuracy: 0.5911094079325542
🔹 Classification Report:
               precision    recall  f1-score   support

           0       0.73      0.46      0.56      1283
           1       0.69      0.15      0.25       292
           2       0.86      0.53      0.65      1645
           3       0.50      0.88      0.63      3311
           4       0.75      0.52      0.61       675
           5       0.62      0.47      0.53      2015
           6       0.85      0.94      0.89        36
           7       0.68      0.35      0.46      1181

    accuracy                           0.59     10438
   macro avg       0.71      0.54      0.58     10438
weighted avg       0.65      0.59      0.58     10438



In [ ]:
import pandas as pd
import re
import string
import neattext.functions as nfx
from textblob import Word
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib

# Load dataset
df = pd.read_csv("emotion_dataset_raw.csv")

# Initialize helpers
lemmatizer = WordNetLemmatizer()

# Text cleaning function
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'https?://\S+|www\.\S+', '', text)  # Remove URLs
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    text = nfx.remove_userhandles(text)  # Remove @mentions
    text = " ".join([Word(word).lemmatize() for word in text.split()])  # Lemmatization
    return text

# Apply cleaning
df['Clean_Text'] = df['Text'].apply(clean_text)

# Features and labels
X = df['Clean_Text']
y = df['Emotion']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=12000, ngram_range=(1,2), sublinear_tf=True)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

print("✅ Data Preprocessing Complete")


✅ Data Preprocessing Complete


In [12]:
# Initialize Random Forest
rf_model = RandomForestClassifier(n_estimators=200, max_depth=30, random_state=42, n_jobs=-1)

# Train model
rf_model.fit(X_train_tfidf, y_train)

print("✅ Random Forest Model Trained Successfully!")


✅ Random Forest Model Trained Successfully!


In [13]:
# Make predictions
y_pred = rf_model.predict(X_test_tfidf)

# Evaluate model
accuracy = accuracy_score(y_test, y_pred)
print("🔹 Accuracy:", accuracy)
print("🔹 Classification Report:\n", classification_report(y_test, y_pred))


🔹 Accuracy: 0.46474420387047327
🔹 Classification Report:
               precision    recall  f1-score   support

       anger       0.91      0.18      0.30      1283
     disgust       0.00      0.00      0.00       292
        fear       0.96      0.35      0.51      1645
         joy       0.38      0.99      0.54      3311
     neutral       0.93      0.19      0.31       675
     sadness       0.91      0.22      0.35      2015
       shame       0.00      0.00      0.00        36
    surprise       0.90      0.17      0.28      1181

    accuracy                           0.46     10438
   macro avg       0.62      0.26      0.29     10438
weighted avg       0.72      0.46      0.41     10438



C:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [14]:
# Example usage
text = "I am feeling very happy today!"
print("Predicted Emotion:", predict_emotion(text))

Predicted Emotion: joy
